In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from riptide.data.UCF101.dataset import train_dataset, test_dataset
#tf.logging.set_verbosity(tf.logging.ERROR)

/opt/conda/envs/riptide-py3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def model_fn(features, labels, mode):
    i3d = hub.Module("https://tfhub.dev/deepmind/i3d-kinetics-400/1")
    logits = i3d(features)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    # otherwise need to compute some more
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    # configure train op if we need to
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        #train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        train_op = tf.no_op()
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    # otherwise we must be doing eval
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [3]:
def train_input_fn():
    ds = train_dataset()
    ds = ds.cache().shuffle(buffer_size=5000).batch(1)
    ds = ds.prefetch(tf.contrib.data.AUTOTUNE)
    ds = ds.repeat()
    return ds

def eval_input_fn():
    ds = test_dataset()
    ds = ds.batch(1)
    return ds

In [4]:
testimator = tf.estimator.Estimator(model_fn=model_fn, model_dir="/data/jwfromm/models/test")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/data/jwfromm/models/test', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2299478898>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [5]:
testimator.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/RGB/inception_i3d/Conv3d_1a_7x7/batch_norm/beta:0 from checkpoint b'/tmp/tfhub_modules/092225fb776e28d6d64ac605ab6be03f18dd2027/variables/variables' with RGB/inception_i3d/Conv3d_1a_7x7/batch_norm/beta
INFO:tensorflow:Initialize variable module/RGB/inception_i3d/Conv3d_1a_7x7/batch_norm/moving_mean:0 from checkpoint b'/tmp/tfhub_modules/092225fb776e28d6d64ac605ab6be03f18dd2027/variables/variables' with RGB/inception_i3d/Conv3d_1a_7x7/batch_norm/moving_mean
INFO:tensorflow:Initialize variable module/RGB/inception_i3d/Conv3d_1a_7x7/batch_norm/moving_variance:0 from checkpoint b'/tmp/tfhub_modules/092225fb776e28d6d64ac605ab6be03f18dd2027/variables/variables' with RGB/inception_i3d/Conv3d_1a_7x7/batch_norm/moving_variance
INFO:tensorflow:Initialize variable module/RGB/inception_i3d/Conv3d_1a_7x7/conv_3d/w:0 from checkpoint b'/tmp/tfhub_modules/092225fb776e

KeyboardInterrupt: 